# Generate Data: Era 1

## Table of Contents
1. Summary
2. Pre-Requisite Knowledge
3. Setup [Installing Dependencies, Importing Libraries, and Notebook Parameters]
4. Retrieve Annotated Detection Metadata
5. Download & Splice Clips (.wav)

## Summary:
We utilize the OrcaHello tool to detect SRKW calls and save these detections for review and annotation by our team of experienced moderators (via the OrcaHello portal).
This notebook generates a local dataset of these annotated detections for model training.

## Pre-Requisite Knowledge:
* The SRKW call detector assigns call confidence score per 1-second intervals, yet the detections are aggregated and annotated (by our moderator team) at 60 second intervals. We use the following terminology to disambiguate "detection": the 60-second detection is called a segment and the 1-second detection is called a clip.
* Since OrcaHello only allows moderators to annotate segments and not clips, the clip labels must be inferred. In cases of false detections (i.e. false positive), every clip in the segment is known to be false positive. In cases of true detections (i.e. true positive), at least one clip in the segment is known to be true positive. Thus, it's critical to note that true positive labels are noisy; in other words, some of the clips in true positive segments can be false detections.
* Era 1 covers all OrcaHello-annotated detections between January 1, 2020 and December 31, 2024.
* Era 1 dataset contains many examples of true positive and false positive SRKW calls. It however contains no examples of true negative or false negative SRKW calls. For inclusion of additional true negative examples, consider Era 0 dataset. For inclusion of additional false negative examples, consider searching through segments for undetected calls or searching through stored segments (before or after) given annotated segments.

## Installing Dependencies

In [1]:
import sys
!{sys.executable} -m pip install -r ../requirements.txt

  Cloning https://github.com/kevinbird15/fastaudio.git (to revision fastaudio_refresh) to /tmp/pip-req-build-8ehevsvf
  Running command git clone --filter=blob:none --quiet https://github.com/kevinbird15/fastaudio.git /tmp/pip-req-build-8ehevsvf
  Running command git checkout -b fastaudio_refresh --track origin/fastaudio_refresh
  Switched to a new branch 'fastaudio_refresh'
  branch 'fastaudio_refresh' set up to track 'origin/fastaudio_refresh'.
  Resolved https://github.com/kevinbird15/fastaudio.git to commit 045a276d227387c630b6aa1a3eba6fae1f6646c4
  Preparing metadata (setup.py) ... done


## Importing Libraries

In [2]:
from argparse import Namespace
from datetime import date
import gc
import librosa
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from pathlib import Path
from pydub import AudioSegment
import sys
from tqdm.notebook import tqdm_notebook
import wave

# Ensure that matplotlib-generated charts are shown in notebook
%matplotlib inline

# Notebook Parameters

In [3]:
REPO_ROOT = Path.cwd().parent

## Import Local Libraries

In [4]:
sys.path.append(str(REPO_ROOT))

import orca_ml.generate_clip_dataset
import orca_ml.retrieve_detection_metadata

# Retrieve Annotated Detection Metadata

##  Retrieve Train Set Metadata

In [12]:

from importlib import reload
reload(orca_ml.retrieve_detection_metadata)
import orca_ml.retrieve_detection_metadata

arg_metadata_file = REPO_ROOT / "data/metadata/train/metadata.json"
arg_start_date = date.fromisoformat("2020-01-01")
arg_end_date = date.fromisoformat("2022-02-28")
arg_max_batch = 100 # MANUALLY VALIDATED MAX NUMBER OF PAGES NEEDED FOR THESE DATES

args = Namespace(metadata_file=arg_metadata_file, start_date=arg_start_date, end_date=arg_end_date, max_batch=arg_max_batch)

orca_ml.retrieve_detection_metadata.retrieve(args)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:42<00:00,  2.36it/s]


## Retrieve Validation Set Metadata

In [ ]:
arg_metadata_file = REPO_ROOT / "data/metadata/val/metadata.json"
arg_start_date = date.fromisoformat("2022-03-01")
arg_end_date = date.fromisoformat("2022-12-31")
arg_max_batch = 50 # MANUALLY VALIDATED MAX NUMBER OF PAGES NEEDED FOR THESE DATES

args = Namespace(metadata_file=arg_metadata_file, start_date=arg_start_date, end_date=arg_end_date, max_batch=arg_max_batch)

orca_ml.retrieve_detection_metadata.retrieve(args)

## Retrieve Test Set Metadata

In [ ]:
arg_metadata_file = REPO_ROOT / "data/metadata/test/metadata.json"
arg_start_date = date.fromisoformat("2023-01-01")
arg_end_date = date.fromisoformat("2024-12-31")
arg_max_batch = 30 # MANUALLY VALIDATED MAX NUMBER OF PAGES NEEDED FOR THESE DATES

args = Namespace(metadata_file=arg_metadata_file, start_date=arg_start_date, end_date=arg_end_date, max_batch=arg_max_batch)

orca_ml.retrieve_detection_metadata.retrieve(args)

# Download and Splice Clips (*.wav)

## Download and Splice Train Set Clips

In [5]:
arg_metadata_file = REPO_ROOT / "data/metadata/train/metadata.json"
arg_clip_folder = REPO_ROOT / "data/processed/train"
arg_top_k = 60
arg_round_name = "OfficialEra1"

args = Namespace(metadata_file=arg_metadata_file, clip_folder=arg_clip_folder, top_k=arg_top_k, round_name=arg_round_name)

orca_ml.generate_clip_dataset.generate(args)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4195/4195 [39:05<00:00,  1.79it/s]


## Download and Splice Validation Set Clips

In [7]:
arg_metadata_file = REPO_ROOT / "data/metadata/val/metadata.json"
arg_clip_folder = REPO_ROOT / "data/processed/val"
arg_top_k = 60
arg_round_name = "OfficialEra1"

args = Namespace(metadata_file=arg_metadata_file, clip_folder=arg_clip_folder, top_k=arg_top_k, round_name=arg_round_name)

orca_ml.generate_clip_dataset.generate(args)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1580/1580 [19:29<00:00,  1.35it/s]


## Download and Splice Test Set Clips

In [8]:
arg_metadata_file = REPO_ROOT / "data/metadata/test/metadata.json"
arg_clip_folder = REPO_ROOT / "data/processed/test"
arg_top_k = 60
arg_round_name = "OfficialEra1"

args = Namespace(metadata_file=arg_metadata_file, clip_folder=arg_clip_folder, top_k=arg_top_k, round_name=arg_round_name)

orca_ml.generate_clip_dataset.generate(args)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1324/1324 [11:04<00:00,  1.99it/s]
